In [4]:
import json,csv
from kafka import KafkaProducer
import time

In [5]:
def producer(file,sleep_time=0.00015):
    producer = KafkaProducer( bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
                               value_serializer=lambda v: json.dumps(v).encode('utf-8'))
    path ='./Data/'
    with open(path+file+'.csv') as file:
        reader = csv.DictReader(file, delimiter=",")
        i = 0
        start = time.time()
        for row in reader:
            
            row["HEAD"] = int(row["HEAD"])
            row["FPGA"] = int(row["FPGA"])
            row["TDC_CHANNEL"] = int(row["TDC_CHANNEL"])
            row["ORBIT_CNT"] = float(row["ORBIT_CNT"])
            row["BX_COUNTER"] = int(row["BX_COUNTER"])
            row["TDC_MEAS"] = float(row["TDC_MEAS"])
            producer.send(topic='Experiment_measurements', value=row, )
            #producer.send(topic='Experiment_measurements', value=json.dumps(row))
            #producer.flush()
            time.sleep(sleep_time) #This was adjusted "by eye" to send 1000 elements/second
            i += 1
            if (i==1000):
                i = 0
                end = time.time()
                print(f"The time to print elements is:",end-start)
                start = end
                

In [6]:
KAFKA_BOOTSTRAP_SERVERS = 'slave01:9092'

for file in ['data_000000']:
    print(file)
    producer(file)

data_000000
The time to print elements is: 0.8660340309143066
The time to print elements is: 0.8392119407653809
The time to print elements is: 0.9114000797271729
The time to print elements is: 0.9376523494720459
The time to print elements is: 1.108109712600708
The time to print elements is: 0.860774040222168
The time to print elements is: 0.7988495826721191
The time to print elements is: 0.9045853614807129
The time to print elements is: 0.8702020645141602
The time to print elements is: 0.9040765762329102
The time to print elements is: 0.9650232791900635
The time to print elements is: 0.9408128261566162
The time to print elements is: 0.8581876754760742
The time to print elements is: 0.8384909629821777
The time to print elements is: 0.851900577545166
The time to print elements is: 0.9142837524414062
The time to print elements is: 1.0082783699035645
The time to print elements is: 0.9165661334991455
The time to print elements is: 0.8655598163604736
The time to print elements is: 0.81596279

KeyboardInterrupt: 

In [ ]:
producer.close()